## Importar librerias

In [1]:
import numpy as np
import math
import re # expresiones regulares
import pandas as pd
from bs4 import BeautifulSoup # para trabajar texto en varios formatos

#from google.colab import drive

In [2]:
# El código anterior me trae la última versión de keras
try:
  %tensorflow_version 2.x
except Exception:
  pass
  
import tensorflow as tf

from tensorflow.keras import layers # importar layers desde keras

import tensorflow_datasets as tfds # este me trae los tokens

2022-08-26 15:51:52.480445: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-26 15:51:52.480502: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Cargar conjunto de datos

In [3]:
#cols=["resultado","mensaje"]
data = pd.read_csv("SMSSpamCollection(Filtrada).csv", on_bad_lines='skip', sep=';')

In [4]:
data

,resultado,mensaje
0,spam,Free entry in 2 a wkly comp to win FA Cup fina...
1,spam,FreeMsg Hey there darling it's been 3 week's n...
2,ham,As per your request 'Melle Melle (Oru Minnamin...
3,spam,WINNER!! As a valued network customer you have...
4,spam,Had your mobile 11 months or more? U R entitle...
...,...,...
1285,spam,Had your contract mobile 11 Mnths? Latest Moto...
1286,ham,Yeh. Indians was nice. Tho it did kane me off ...
1287,spam,REMINDER FROM O2: To get 2.50 pounds free call...
1288,spam,This is the 2nd time we have tried 2 contact u...


In [5]:
data['resultado'] = data['resultado'].apply(lambda x:1 if x == 'ham' else 0) 

In [6]:
data

,resultado,mensaje
0,0,Free entry in 2 a wkly comp to win FA Cup fina...
1,0,FreeMsg Hey there darling it's been 3 week's n...
2,1,As per your request 'Melle Melle (Oru Minnamin...
3,0,WINNER!! As a valued network customer you have...
4,0,Had your mobile 11 months or more? U R entitle...
...,...,...
1285,0,Had your contract mobile 11 Mnths? Latest Moto...
1286,1,Yeh. Indians was nice. Tho it did kane me off ...
1287,0,REMINDER FROM O2: To get 2.50 pounds free call...
1288,0,This is the 2nd time we have tried 2 contact u...


## Limpieza de datos

In [7]:
def clean_tweet(tweet):

  tweet= BeautifulSoup(tweet,"lxml").get_text() # sirve para trabajar en un formato especial de tratar el texto. hay que usar el get_text para que me lo devuela en un formato legible
  
  # Dejamos todo en minúscula
  tweet = tweet.lower()
  # Reemplazamos puntos por espacios
  tweet = tweet.replace('.',' ')
  # Eliminamos la @ y su mención
  tweet = re.sub(r"@[A-Za-z0-9.]+",' ',tweet) # después del @ puede leer lo que se encuentra en el []. El + significa que puede ser mas de un caracter a los que se hace referencia. 
  # El r significa raw o string y le dice a la sentencia que lea la línea tal cual, por ejemplo si vieniera con hipervinculo que no lo considere.
  # Eliminamos los links de la URLs
  tweet = re.sub(r"https?://[A-Za-z0-9./]+",' ', tweet) # aqui el ? me dice que el caracter anterior puede estar o no
  # Nos quedamos solo con los caracteres
  tweet = re.sub(r"[^a-zA-Z.!?']",' ',tweet)# cualquier cosa que no sea lo que viene después de ^ se va a sustituir por espacios en blanco
  # Eliminamos los sitios web
  tweet = re.sub(r"www[A-Za-z0-9./]+",' ', tweet)
  # Eliminamos espacios en blanco adicionales
  tweet = re.sub(r" +",' ',tweet)# si existe más de un espacio en blanco lo reemplazamos por uno solo. El más me dice eso, si hay más de uno reemplazo
  return tweet

In [8]:
data_clean = [clean_tweet(tweet) for tweet in data.mensaje] # esto me queda en una lista guardado

/home/jupyter/.local/lib/python3.10/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [9]:
set(data.resultado) 

{0, 1}

## Tokenización

In [11]:
# El corpus es la lista del texto que se va a analizar|

tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    data_clean,target_vocab_size = 2**16
)

data_inputs = [tokenizer.encode(sentence) for sentence in data_clean]

## Padding

In [12]:
MAX_LEN = max(len(sentence) for sentence in data_inputs)
data_inputs = tf.keras.preprocessing.sequence.pad_sequences(
    data_inputs,
    value = 0,
    padding = "post",
    maxlen = MAX_LEN
)

## Dividimos la mjuestra en train y test

In [13]:
y = pd.DataFrame(data.resultado)
x = pd.DataFrame(data_inputs)

In [25]:
x

,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,39,40,41,42,43
0,24,216,13,2,383,561,1,79,1198,1001,...,0,0,0,0,0,0,0,0,0,0
1,426,178,131,1542,161,4868,25,83,221,4868,...,4471,1,55,1,1841,0,0,0,0,0
2,64,110,9,922,179,825,825,1386,1425,1393,...,0,0,0,0,0,0,0,0,0,0
3,691,408,64,2,481,214,139,3,21,83,...,0,0,0,0,0,0,0,0,0,0
4,138,9,50,820,17,819,15,7,74,670,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1285,138,9,1220,50,1961,15,154,491,81,669,...,0,0,0,0,0,0,0,0,0,0
1286,1254,3814,67,395,787,28,130,3723,19,252,...,0,0,0,0,0,0,0,0,0,0
1287,1828,30,394,1,36,334,24,10,748,8,...,0,0,0,0,0,0,0,0,0,0
1288,40,11,5,199,89,34,21,330,96,7,...,0,0,0,0,0,0,0,0,0,0


In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x,y,test_size = 0.3,random_state = 12)

## Construcción del modelo

In [15]:
class DCNN(tf.keras.Model):
  
  def __init__(self,
             vocab_size,
             emb_dim = 128,
             nb_filters = 50, 
             FFN_units = 512,
             nb_classes = 2,
             dropout_rate = 0.1,
             training = False,
             name = "dcnn"):
    
    super(DCNN, self).__init__(name=name)

    self.embedding = layers.Embedding(vocab_size,
        emb_dim)
    
    self.bigram = layers.Conv1D(filters = nb_filters,
    kernel_size = 2,
    padding = "valid",
    activation = "relu")
    
    self.trigram = layers.Conv1D(filters = nb_filters,
                                kernel_size = 3,
                                padding = "valid",
                                activation = "relu")
     
    self.fourgram = layers.Conv1D(filters = nb_filters,
                                kernel_size = 4,
                                padding = "valid",
                                activation = "relu")
    
    self.pool = layers.GlobalMaxPool1D()


    self.dense_1 = layers.Dense(units = FFN_units, activation = "relu")
    self.dropout = layers.Dropout(rate = dropout_rate)
    if nb_classes == 2:
        self.last_dense = layers.Dense(units = 1, activation = "sigmoid")
    else:
      self.last_dense = layers.Dense(units = nb_classes,activation = "softmax") 

  def call(self,inputs, training):
    x = self.embedding(inputs)
    x_1 = self.bigram(x)
    x_1 = self.pool(x_1)       
    x_2 = self.trigram(x)
    x_2 = self.pool(x_2)         
    x_3 = self.fourgram(x)
    x_3 = self.pool(x_3)

    merged = tf.concat([x_1,x_2,x_3], axis = -1)# con el -1 le digo el último eje de la combinacion // batch_size,3*nb_filters
    merged = self.dense_1(merged)
    merged = self.dropout(merged, training)
    output = self.last_dense(merged)

    return output             
 #vocab_size,# tamaño de las palabras a trabajar
  #           emb_dim = 128,# a que espacio vectorial vamos a llevar nuestras palabras (cada una de ellas se irá a un espacio de 128 dimensiones)
   #          nb_filters = 50, # cuantas palabras filtra
    #         FNN_units 512,# numero de neuronas
     #        nb_classes = 2,# categorias de clasificacion
  #dropout_rate = 0.1, evitar el overfitting, el 10% de las neuronas no aprenderan transmitiran lo aprendido

## Aplicación

### Configuración de parámetros

In [16]:
VOCAB_SIZE = tokenizer.vocab_size

EMB_DIM = 200
NB_FILTERS = 100
FFN_UNITS = 256
NB_CLASSES = 2 #LEN(SET(TRAIN_LABELS))

DROPOUT_RATE = 0.4 # razón de no aprendizaje de las neuronas
BATCH_SIZE = 32
NB_EPOCHS = 2

### Entrenamiento

In [17]:
Dcnn = DCNN(vocab_size = VOCAB_SIZE,
            emb_dim = EMB_DIM,
            nb_filters = NB_FILTERS,
            FFN_units = FFN_UNITS,
            nb_classes = NB_CLASSES,
            dropout_rate = DROPOUT_RATE)

2022-08-26 15:51:57.243969: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-26 15:51:57.244003: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-26 15:51:57.244030: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jupyter): /proc/driver/nvidia/version does not exist
2022-08-26 15:51:57.244311: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [18]:
if NB_CLASSES == 2:
  Dcnn.compile (loss = "binary_crossentropy",
                optimizer = "adam",
                metrics = ["accuracy"])
else:
  Dcnn.compile(loss= "sparse_categorical_crossentropy",
               optimizer = "adam",
               metrics = ["sparse_categorical_accuracy"])  

In [19]:
#checkpoint_path = '/content/drive/MyDrive/Colab Notebooks/CNN Spam Ham/check_point'

#ckpt = tf.train.Checkpoint(Dcnn = Dcnn)

#ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep = 10)

#if ckpt_manager.latest_checkpoint:
#  ckpt.restore(ckpt_manager.lastest_checkpoint)
 # print("Último checkpoint restaurado")

In [20]:
Dcnn.fit(X_train,
         Y_train,
         batch_size = BATCH_SIZE,
         epochs = NB_EPOCHS)
#ckpt_manager.save()

Epoch 1/2
29/29 [==============================] - 1s 18ms/step - loss: 0.6075 - accuracy: 0.7807
Epoch 2/2
29/29 [==============================] - 0s 16ms/step - loss: 0.1765 - accuracy: 0.9690


## Evaluación

In [21]:
results = Dcnn.evaluate(X_test,Y_test,batch_size = BATCH_SIZE)
print(results)

13/13 [==============================] - 0s 5ms/step - loss: 0.1124 - accuracy: 0.9767
[0.11240389198064804, 0.9767441749572754]


In [22]:
Dcnn(np.array([tokenizer.encode("https: you win game")]),training = False).numpy()

array([[0.30338168]], dtype=float32)

In [23]:
#from sklearn.metrics import classification_report

#clf = grid_search.best_estimator_
#pred_lda = clf.predict(X_test)

#print(classification_report(Y_test, pred_lda))